In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# read the data
data = pd.read_csv("customer_value_analysis.csv")
display(data.head)

In [ ]:
# define function to create countplot for each categorical column

def categorical_plot(dataframe):
    """plots categorical columns in a dataframe
        input: dataframe
        output seaborn countplot"""
    for column in dataframe.select_dtypes('object').columns:
        order = dataframe[column].value_counts().index
        if dataframe[column].nunique() >=6:
            sns.countplot(data=dataframe, y=column, order = order)
        else: sns.countplot(data=dataframe, x = column, order = order)
        plt.show()
        
categorical_plot(data)
        

In [ ]:
print(data.columns)

In [ ]:
# drop and transform columns
data.drop(columns=['Policy Type'], inplace=True)


In [ ]:
# inspecting the unique values
unique_values = data['Policy'].unique()
print(unique_values)

In [ ]:
# function to transform values in Policy
def transform_policy(value):
    if 'L1' in value:
        return 'L1'
    elif 'L2' in value:
        return 'L2'
    else:
        return 'L3'
data['Policy'] = data['Policy'].apply(transform_policy)

In [ ]:
#checking after the function was applied
unique_values = data['Policy'].unique()
print(unique_values)

In [ ]:
# plot the time series
plt.figure(figsize=(10,10))
data['Effective To Date'] = pd.to_datetime(data['Effective To Date'])
sns.lineplot(data=data, x='Effective To Date', y='Total Claim Amount')
plt.show()

In [ ]:
# analysis of effective_to_date
time_df = pd.DataFrame()
time_df['day'] = data['Effective To Date'].dt.day
time_df['day_name'] = data['Effective To Date'].dt.day_name()
time_df['week'] = data['Effective To Date'].dt.week
time_df['month'] = data['Effective To Date'].dt.month_name()
time_df['total_claim_amount'] = data['Total Claim Amount']

time_df.head()

In [ ]:
# analysis by day
correct_days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_analysis = time_df.groupby('day_name').total_claim_amount.mean().round(2).reindex(correct_days_order)
sns.lineplot(x=day_analysis.index, y=day_analysis.values)
plt.show()


In [ ]:
# claims by day name
claims_by_day = time_df['day_name'].value_counts().reindex(correct_days_order)
print(claims_by_day)

# median tgt by day
median_target_by_day = time_df.groupby('day_name').total_claim_amount.median().sort_values(ascending=False)
sns.barplot(x=median_target_by_day.index, y=median_target_by_day.values)
plt.show()

# total claims by dau
claims_by_day = time_df['day_name'].value_counts().reindex(correct_days_order)
print(claims_by_day)





The claims by day are uniform.

In [ ]:
# compute total target column aggregated month rounded to two decimals and then reorder the index of the resulting
# pandas series using .reindex(index=list_of_correct_values)
correct_months_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
month_analysis = time_df.groupby('month').total_claim_amount.sum().round(2).reindex(correct_months_order)

In [ ]:
#size of the plot
plt.figure(figsize=(12, 6))

#bar plot
sns.barplot(x=month_analysis.index, y=month_analysis.values)

# Giving the plot a title and labels for clarity
plt.title('Total Claim Amount by Month')
plt.ylabel('Total Claim Amount')
plt.xlabel('Month')

# Rotating the month names for better visibility
plt.xticks(rotation=45)

# Displaying the plot
plt.show()


monthly analysis is not so interesting because we only have 2 months of data which is too small a set to extrapolate trends or good analysis.

In [ ]:
# function to remove outliers

def remove_outliers(df, col, thr=3):
  """Remove outliers from a DataFrame column based on IQR.

  input:
    df: pandas DataFrame.
    col: column name to remove outliers from.
    thr: threshold multiplier for IQR.

  output:
       pandas DataFrame with the outliers removed.
  """

  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)
  iqr = q3 - q1
  lower_limit = q1 - thr * iqr
  upper_limit = q3 + thr * iqr
  return df.loc[(df[col] >= lower_limit) & (df[col] <= upper_limit)]

# remove outliers from all numeric columns in the df
continuous_clean_df = data.select_dtypes(include=[np.number])
for col in continuous_clean_df.columns:
  continuous_clean_df = remove_outliers(continuous_clean_df, col)

In [ ]:
display(continuous_clean_df)


15. Concatenate the continuous_cleaned_df, discrete_df, categorical_df, and the relevant column of time_df. After removing outliers the continuous_cleaned dataframe will have fewer rows (when you concat the individual dataframes using pd.concat()) the resulting dataframe will have NaN's because of the different sizes of each dataframe. Use pd.dropna() and .reset_index() to fix the final dataframe.

The instructions above make no sense to me as we did not generate a 'discrete_df' or a 'categorical_df' so the code underneath will not work.

In [ ]:
final_df = pd.concat([continuous_clean_df, discrete_df, categorical_df, time_df], axis=1).dropna().reset_index(drop=True)

# Reorder columns
columns_order = [col for col in final_df.columns if col != 'total_claim_amount'] + ['total_claim_amount']
final_df = final_df[columns_order]